# 구글 드라이브 마운트 및 경로 이동

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd '/content/drive/MyDrive/cakd7/3차프로젝트_dl_1025_1128/5팀(윤정준, 최애림, 진청아, 이정현a)/코드'

/content/drive/.shortcut-targets-by-id/1SSMjIaDcwniFaLUaXBuU9Ry1IjYbaq-j/cakd7/3차프로젝트_dl_1025_1128/5팀(윤정준, 최애림, 진청아, 이정현a)/코드


# import

In [ ]:
!pip install tensorflow_addons
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow.keras import layers, optimizers, activations, metrics, models, losses
from tensorflow.keras import Input
from keras.applications.xception import Xception
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from keras.layers import concatenate
from keras.models import Model
import keras
import tensorflow as tf
from keras.callbacks import Callback
from seqeval.metrics import f1_score, classification_report
import tensorflow_addons as tfa
from keras.applications.efficientnet_v2 import EfficientNetV2L
import os
import cv2
from tensorflow.keras.utils import array_to_img
from keras.applications.nasnet import NASNetLarge, NASNetMobile

# 1. 모델 생성
- 1) 이미지 데이터 _ 비정형 데이터
- 2) 컬럼 데이터 _ 정형 데이터
- 3) 이미지 데이터 + 컬럼 데이터 모델 concat

### 사용 모델 선언

### 1) 이미지 데이터 
- 1. NASNetMobile
- 2. Xception

#### 1. NASNet

In [ ]:
nasnet_mobile = NASNetMobile()
nasnet_large = NASNetLarge()

In [ ]:
img_input = layers.Input(shape=(480,480,3))
img_dence = layers.Conv2D(128,5,activation='relu' )(img_input)
img_dence = layers.MaxPooling2D()(img_dence)
img_dence = layers.Conv2D(64,5,activation='relu' )(img_dence)
img_dence = layers.Resizing(224,224)(img_dence)
img_dence = layers.Conv2D(3,3,padding='same',activation='relu' )(img_dence)

# 1. nasnet_mobile 모델
img_dence = nasnet_mobile(img_dence)

## 2. nasnet_large 모델
# img_dence = nasnet_large(img_dence)

img_output = layers.Dense(1,activation='sigmoid')(img_dence)

In [ ]:
img_model = keras.Model(img_input, img_output, name='img')
img_model.summary()

Model: "img"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 480, 480, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 476, 476, 128)     9728      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 238, 238, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 234, 234, 64)      204864    
                                                                 
 resizing (Resizing)         (None, 331, 331, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 331, 331, 3)       1731      
                                                               

#### 2. Xception

In [ ]:
# xception = Xception(include_top=True,
#     weights=None,
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation='softmax')

# img_input = layers.Input(shape=(299,299,3))
# img_dence = xception(img_input)
# img_output = layers.Dense(1,activation='sigmoid')(img_dence)

# img_model = keras.Model(img_input, img_output, name='img')
# img_model.summary()

### 2) 컬럼 데이터 

In [ ]:
tabular_input = layers.Input(shape=(23,))
x = layers.Dense(256,activation='tanh')(tabular_input)
x = layers.Dense(512,activation='tanh')(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256,activation='tanh')(x)
x = layers.BatchNormalization()(x)

tabular_dence = layers.Flatten()(x)
tabular_output = layers.Dense(1,activation='sigmoid')(tabular_dence)

In [ ]:
tabular_model = keras.Model(tabular_input, tabular_output, name='tabular')
tabular_model.summary()

Model: "tabular"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 23)]              0         
                                                                 
 dense_1 (Dense)             (None, 256)               6144      
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                           

### 3) 모델 concat

In [ ]:
concat = concatenate([img_dence, tabular_dence])
answer = layers.Dense(1, activation='sigmoid')(concat)
model = Model(inputs=[img_input, tabular_input], outputs=answer)

# 2. 모델 compile

### compile 
- optimizer
  -  RMSprop
  -  Adam
  - NAdam
    - Nadam train data에서 가장 높은 정확도를 보였으면 test data에서도 가장 높은 정확도를 보임

- NAG - > optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True)
- momentum   - > optimizers.SGD(lr=0.1, momentum=0.9)

In [ ]:
# 재현율, 정확도, f1 스코어를 위한 사용자 함수생성
from keras import backend as K

# 재현율
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아냄
    # round : 반올림
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더해줌
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall

# 정확도
def precision(y_target, y_pred):
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) 
    y_target_yn = K.round(K.clip(y_target, 0, 1)) 

    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    count_true_positive_false_positive = K.sum(y_pred_yn)

    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    return precision

# f1스코어
def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)

    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())

    return _f1score

In [ ]:
# 모델 컴파일
# 1. rmsprop
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy', precision, recall, f1score])
## 2. adam
# model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy', precision, recall, f1score])
## 3. nadam
# model.compile(optimizer=optimizers.Nadam(), loss='binary_crossentropy', metrics=['accuracy', precision, recall, f1score])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 dense_4 (Dense)                (None, 256)          6144        ['input_4[0][0]']                
                                                                                                  
 dense_5 (Dense)                (None, 512)          131584      ['dense_4[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 512)         2048        ['dense_5[0][0]']                
 alization)                                                                                   

# 3. 데이터 전처리

## 1) 이미지 데이터

### 이미지데이터 경로 설정

In [ ]:
base_dir = './'

train_dir = os.path.join(base_dir,'train_imgs' )

test_dir = os.path.join(base_dir,'test_imgs' )


train_dir_last = []
for i in os.listdir(train_dir):
  train_dir_last.append(os.path.join(train_dir, i))

train_dir_last = sorted(train_dir_last)


test_dir_last = []
for i in os.listdir(test_dir):
  test_dir_last.append(os.path.join(test_dir, i))

test_dir_last = sorted(test_dir_last)

train_dir_last_re = []
for i in train_dir_last:
    i = i.replace('\\','/')
    train_dir_last_re.append(i)

test_dir_last_re = []
for i in test_dir_last:
    i = i.replace('\\','/')
    test_dir_last_re.append(i)

### 데이터 이미지 변경하여 부르기

In [ ]:
# 학습용 이미지
train_img_list = []
for i in range(len(train_dir_last)):

  img_ = cv2.imread(train_dir_last[i])
  # 이미지 사이즈 nasnet에 맞게 480, 480 으로 변경
  train_img_list.append(cv2.resize(img_.astype('float32'), dsize=(480, 480)))
train_img_list = np.array(train_img_list)

# 테스트용 이미지
test_img_list = []
for i in range(len(test_dir_last)):
  img_ = cv2.imread(test_dir_last[i])
  test_img_list.append(cv2.resize(img_.astype('float32'), dsize=(480, 480)))
test_img_list = np.array(test_img_list)

In [ ]:
# 이미지 저장
# import pickle

# with open('./array/train_img_nasnet_array_canny.pickle', 'wb') as f:
#     pickle.dump(train_img_nasnet_array_canny, f, pickle.HIGHEST_PROTOCOL)

# with open('./array/test_img_nasnet_array_canny.pickle', 'wb') as f:
#     pickle.dump(test_img_nasnet_array_canny, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# 이미지 부르기
import pickle

with open('./array/train_img_nasnet_array_canny.pickle', 'rb') as f:
    train_img_nasnet_array_canny = pickle.load(f)

with open('./array/test_img_nasnet_array_canny.pickle', 'rb') as f:
    test_img_nasnet_array_canny = pickle.load(f)

## 2) 정형 데이터

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df['암의 장경'] = train_df['암의 장경'].fillna(train_df['암의 장경'].mean())
train_df['KI-67_LI_percent'] = train_df['KI-67_LI_percent'].fillna(train_df['KI-67_LI_percent'].mean())
train_df['ER_Allred_score'] = train_df['ER_Allred_score'].fillna(train_df['ER_Allred_score'].mean())
train_df['PR_Allred_score'] = train_df['PR_Allred_score'].fillna(train_df['PR_Allred_score'].mean())
train_df['HER2_SISH_ratio'] = train_df['HER2_SISH_ratio'].fillna(train_df['HER2_SISH_ratio'].mean())
train_df = train_df.fillna(0)
test_df['암의 장경'] = test_df['암의 장경'].fillna(test_df['암의 장경'].mean())
test_df['KI-67_LI_percent'] = test_df['KI-67_LI_percent'].fillna(test_df['KI-67_LI_percent'].mean())
test_df['ER_Allred_score'] = test_df['ER_Allred_score'].fillna(test_df['ER_Allred_score'].mean())
test_df['PR_Allred_score'] = test_df['PR_Allred_score'].fillna(test_df['PR_Allred_score'].mean())
test_df['HER2_SISH_ratio'] = test_df['HER2_SISH_ratio'].fillna(test_df['HER2_SISH_ratio'].mean())
test_df = test_df.fillna(0)

## 3) 라벨 값 생성

In [ ]:
img_label_ = []
for i in train_dir_last_re:
    img_label_.append(train_df[train_df.img_path == i]['N_category'].values[0])
img_label_ = np.array(img_label_)
img_label_.shape

(1000,)

## 4) 학습용, 검증용 데이터 분리

In [ ]:
train_df, val_df, train_labels, val_labels = train_test_split(
                                                    train_df.drop(columns=['N_category'], axis = 1), 
                                                    train_df['N_category'], 
                                                    test_size=0.2, 
                                                    random_state=256
                                                )

In [ ]:
train_img, val_img, train_img_labels, val_img_labels = train_test_split(
                                                    train_img_efficientnet_v2l_array_canny,img_label_,
                                                    test_size=0.2, 
                                                    random_state=256
                                                )

## 5) 데이터 인코딩

In [ ]:
def get_values(value):
    return value.values.reshape(-1, 1)

# 연속형 데이터 컬럼
numeric_cols = ['나이', '암의 장경', 'ER_Allred_score', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2_SISH_ratio']

# 학습에 사용하지 않는 컬럼
ignore_cols = ['ID', 'img_path', 'mask_path', '수술연월일', 'N_category']

for col in train_df.columns:
    if col in ignore_cols:
        continue
    if col in numeric_cols:
        # 정규화
        scaler = StandardScaler()
        train_df[col] = scaler.fit_transform(get_values(train_df[col]))
        val_df[col] = scaler.transform(get_values(val_df[col]))
        test_df[col] = scaler.transform(get_values(test_df[col]))
    else:
        le = LabelEncoder()
        # 라벨 인코딩
        train_df[col] = le.fit_transform(get_values(train_df[col]))
        val_df[col] = le.transform(get_values(val_df[col]))
        test_df[col] = le.transform(get_values(test_df[col]))

# 이미지 0~1 사이로 전처리
train_img_me = train_img/255.
val_img_me = val_img/255.

In [ ]:
train_df.drop(['ID', 'img_path', 'mask_path', '수술연월일'],axis=1,inplace=True)
val_df.drop(['ID', 'img_path', 'mask_path', '수술연월일'],axis=1,inplace=True)

# 4. 모델 학습

In [ ]:
# 모델 자동 저장 함수

## 저장 경로
modelpath = '/content/drive/MyDrive/수업/3차 프로젝트/modelcheck'

## 저장 조건
mc = tf.keras.callbacks.ModelCheckpoint(
    modelpath, monitor='val_f1score', verbose=1, save_best_only=True,
    save_weights_only=False, mode='max', save_freq='epoch', options=None
)

# 모델 EarlyStopping
es = tf.keras.callbacks.EarlyStopping(patience=15)


# 모델의 개선이 없을 경수 lr 조정함수

rlr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3)

# 모델의 epochs 마다 기록 저장

csvlogger = tf.keras.callbacks.CSVLogger("/content/drive/MyDrive/수업/3차 프로젝트/loglog/xec_df_con.log")

In [ ]:
# 학습 시작
model.fit(x=[train_img_me,train_df],
          y=train_labels,batch_size= 5,shuffle=True, epochs=50,
          validation_data=([val_img_me,val_df],val_labels),
          callbacks=[mc,es,rlr,csvlogger])

Epoch 1/50
160/160 [==============================] - ETA: 0s - loss: 0.6565 - accuracy: 0.6525 - precision: 0.6823 - recall: 0.6987 - f1score: 0.6512
Epoch 1: val_f1score improved from -inf to 0.66812, saving model to /content/drive/MyDrive/수업/3차 프로젝트/modelcheck


160/160 [==============================] - 356s 2s/step - loss: 0.6565 - accuracy: 0.6525 - precision: 0.6823 - recall: 0.6987 - f1score: 0.6512 - val_loss: 0.5549 - val_accuracy: 0.6900 - val_precision: 0.6067 - val_recall: 0.8121 - val_f1score: 0.6681 - lr: 0.0010
Epoch 2/50
160/160 [==============================] - ETA: 0s - loss: 0.5942 - accuracy: 0.6975 - precision: 0.7240 - recall: 0.7132 - f1score: 0.6798
Epoch 2: val_f1score improved from 0.66812 to 0.67198, saving model to /content/drive/MyDrive/수업/3차 프로젝트/modelcheck


160/160 [==============================] - 276s 2s/step - loss: 0.5942 - accuracy: 0.6975 - precision: 0.7240 - recall: 0.7132 - f1score: 0.6798 - val_loss: 0.6277 - val_accuracy: 0.6700 - val_precision: 0.6113 - val_recall: 0.8304 - val_f1score: 0.6720 - lr: 0.0010
Epoch 3/50
160/160 [==============================] - ETA: 0s - loss: 0.5690 - accuracy: 0.7113 - precision: 0.7286 - recall: 0.7209 - f1score: 0.6896
Epoch 3: val_f1score improved from 0.67198 to 0.70325, saving model to /content/drive/MyDrive/수업/3차 프로젝트/modelcheck


160/160 [==============================] - 273s 2s/step - loss: 0.5690 - accuracy: 0.7113 - precision: 0.7286 - recall: 0.7209 - f1score: 0.6896 - val_loss: 0.4884 - val_accuracy: 0.7750 - val_precision: 0.7375 - val_recall: 0.7046 - val_f1score: 0.7033 - lr: 0.0010
Epoch 4/50
160/160 [==============================] - ETA: 0s - loss: 0.5424 - accuracy: 0.7375 - precision: 0.7495 - recall: 0.7703 - f1score: 0.7296
Epoch 4: val_f1score did not improve from 0.70325
160/160 [==============================] - 151s 945ms/step - loss: 0.5424 - accuracy: 0.7375 - precision: 0.7495 - recall: 0.7703 - f1score: 0.7296 - val_loss: 0.5322 - val_accuracy: 0.7500 - val_precision: 0.6858 - val_recall: 0.7192 - val_f1score: 0.6788 - lr: 0.0010
Epoch 5/50
160/160 [==============================] - ETA: 0s - loss: 0.5465 - accuracy: 0.7287 - precision: 0.7448 - recall: 0.7516 - f1score: 0.7138
Epoch 5: val_f1score did not improve from 0.70325
160/160 [==============================] - 151s 946ms/step - 

160/160 [==============================] - 278s 2s/step - loss: 0.5353 - accuracy: 0.7262 - precision: 0.7464 - recall: 0.7431 - f1score: 0.7107 - val_loss: 0.5339 - val_accuracy: 0.7400 - val_precision: 0.6375 - val_recall: 0.8358 - val_f1score: 0.7047 - lr: 0.0010
Epoch 7/50
160/160 [==============================] - ETA: 0s - loss: 0.5122 - accuracy: 0.7425 - precision: 0.7562 - recall: 0.7776 - f1score: 0.7306
Epoch 7: val_f1score did not improve from 0.70474
160/160 [==============================] - 152s 951ms/step - loss: 0.5122 - accuracy: 0.7425 - precision: 0.7562 - recall: 0.7776 - f1score: 0.7306 - val_loss: 0.4785 - val_accuracy: 0.7750 - val_precision: 0.7354 - val_recall: 0.7192 - val_f1score: 0.7041 - lr: 1.0000e-04
Epoch 8/50
160/160 [==============================] - ETA: 0s - loss: 0.5287 - accuracy: 0.7412 - precision: 0.7656 - recall: 0.7621 - f1score: 0.7228
Epoch 8: val_f1score improved from 0.70474 to 0.70915, saving model to /content/drive/MyDrive/수업/3차 프ᄅ

160/160 [==============================] - 277s 2s/step - loss: 0.5287 - accuracy: 0.7412 - precision: 0.7656 - recall: 0.7621 - f1score: 0.7228 - val_loss: 0.4796 - val_accuracy: 0.7800 - val_precision: 0.7562 - val_recall: 0.7067 - val_f1score: 0.7091 - lr: 1.0000e-04
Epoch 9/50
160/160 [==============================] - ETA: 0s - loss: 0.4788 - accuracy: 0.7713 - precision: 0.7865 - recall: 0.8224 - f1score: 0.7700
Epoch 9: val_f1score did not improve from 0.70915
160/160 [==============================] - 152s 951ms/step - loss: 0.4788 - accuracy: 0.7713 - precision: 0.7865 - recall: 0.8224 - f1score: 0.7700 - val_loss: 0.4746 - val_accuracy: 0.7800 - val_precision: 0.7521 - val_recall: 0.7067 - val_f1score: 0.7091 - lr: 1.0000e-04
Epoch 10/50
160/160 [==============================] - ETA: 0s - loss: 0.5045 - accuracy: 0.7638 - precision: 0.7755 - recall: 0.8087 - f1score: 0.7540
Epoch 10: val_f1score improved from 0.70915 to 0.72127, saving model to /content/drive/MyDrive/수업/3

160/160 [==============================] - 278s 2s/step - loss: 0.5045 - accuracy: 0.7638 - precision: 0.7755 - recall: 0.8087 - f1score: 0.7540 - val_loss: 0.4731 - val_accuracy: 0.7850 - val_precision: 0.7400 - val_recall: 0.7400 - val_f1score: 0.7213 - lr: 1.0000e-04
Epoch 11/50
160/160 [==============================] - ETA: 0s - loss: 0.4601 - accuracy: 0.7775 - precision: 0.7937 - recall: 0.8170 - f1score: 0.7715
Epoch 11: val_f1score did not improve from 0.72127
160/160 [==============================] - 152s 948ms/step - loss: 0.4601 - accuracy: 0.7775 - precision: 0.7937 - recall: 0.8170 - f1score: 0.7715 - val_loss: 0.4791 - val_accuracy: 0.7650 - val_precision: 0.7054 - val_recall: 0.7004 - val_f1score: 0.6811 - lr: 1.0000e-04
Epoch 12/50
160/160 [==============================] - ETA: 0s - loss: 0.5162 - accuracy: 0.7375 - precision: 0.7531 - recall: 0.7556 - f1score: 0.7163
Epoch 12: val_f1score did not improve from 0.72127
160/160 [==============================] - 151s 9

## 예측데이터 준비 및 전처리

In [ ]:
test_df.drop(['ID', 'img_path', '수술연월일'],axis=1,inplace=True)

In [ ]:
# 이미지 0 ~ 1 사이로 전처리
test_img_nasnetmobile_array_canny_re = test_img_nasnetmobile_array_canny_re/255.

# 5. 예측

In [ ]:
# 모델 불러오기
best_model = tf.keras.models.load_model('/content/drive/MyDrive/수업/3차 프로젝트/modelcheck',custom_objects={'precision':precision, 'recall':recall, 'f1score':f1score})

In [ ]:
# 예측
pred = best_model.predict(x=[test_img_nasnetmobile_array_canny_re,test_df])

8/8 [==============================] - 19s 1s/step


In [ ]:
# 임계값 설정
## predict 값이 0.5 이상이면 1, 아니면 0
result = []
for i in pred:
    if i >= 0.5:
        result.append(1)
    else:
        result.append(0)

# 6. 제출 준비

In [ ]:
submit = pd.read_csv('./submit.csv')

In [ ]:
submit['N_category'] = result

In [ ]:
submit.to_csv('./sub/submit_nasnetmobile_concat_df_canny.csv',index=False)